In [ ]:
import numpy as np
import pandas as pd

import holoviews as hv; hv.extension('bokeh', logo=False)
import hvplot.pandas

In [ ]:
def box(dat, title=''):
    df = pd.DataFrame( dat, columns=[str(i) for i in range(dat.shape[1])] )
    return df.hvplot.box().opts(title=title)

In [ ]:
n  = 200
L  = 8
x  = np.linspace(0,L,n)
x1 = x[:100]    # Train
x2 = x[100:200] # Test
n1 = len(x1)
n2 = len(x2)

ftrain = np.power(x1,2)     # Train parabola x = [0,4]
ftest  = np.power(x2,2)     # Test parabola x = [4,8]

h=\
hv.Curve((x1,ftrain),label='Training Set').opts(width=500)*\
hv.Curve((x2,ftest),label='Test Set')
h.opts(legend_position="top_left")

In [ ]:
M   = 30 # number of model terms
Eni = np.zeros((100,M))
Ene = np.zeros((100,M))
for jj in range(M):
    phi_i = np.zeros((n1,jj+1))
    phi_e = np.zeros((n2,jj+1))
    for j in range(jj):
        phi_i[:,j] = np.power(x1,j) # interpolation key
        phi_e[:,j] = np.power(x2,j) # extrapolation key
    
    f = np.power(x,2)
    for j in range(100):
        fni       = np.power(x1,2) + 0.1*np.random.randn(n1) # interpolation
        fne       = np.power(x2,2) + 0.1*np.random.randn(n2) # extrapolation

        ani       = np.linalg.pinv(phi_i) @ fni
        fnai      = phi_i @ ani
        Eni[j,jj] = np.linalg.norm(ftrain-fnai,ord=2)/np.linalg.norm(ftrain,ord=2)

        fnae      = phi_e @ ani # use loadings from x in [0,4]
        Ene[j,jj] = np.linalg.norm(ftest-fnae,ord=2)/np.linalg.norm(ftest,ord=2)

In [ ]:
h=\
box(Eni).opts(ylim=(0,0.7))+box(Eni).opts(ylim=(0,0.02))+\
box(Ene).opts(ylim=(0,2e14))+box(np.log(Ene+1)).opts(ylim=(0,30))

h.opts(shared_axes=False).cols(2)